## Arabic Text Categorization Using SANAD Dataset

This notebook implements an Arabic text classification/categorization using a subset of the dataset SANAD. The subset contains **45500** Arabic news articles organized into 7 balanced categories from www.alkhaleej.ae.

## Dataset Overview

The dataset consists of Arabic news articles categorized into the following labels:
- **Culture**
- **Finance**
- **Medical**
- **Politics**
- **Religion**
- **Sports**
- **Tech**

# Import necessary libraries

In [ ]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Getting The Data

In [ ]:
# Function to load data from folders
def load_data_from_folders(main_folder_path):
    data = []
    for category_folder in os.listdir(main_folder_path):
        category_label = category_folder  # Assuming folder name is the category label
        category_path = os.path.join(main_folder_path, category_folder)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'text': content, 'category': category_label})
    return pd.DataFrame(data)

# Main folder containing subfolders for each category
main_folder_path = '/kaggle/input/sanad-dataset'

# Load data from folders
WholeData = load_data_from_folders(main_folder_path)

## More Details About The Data

In [ ]:
WholeData.head(10)

In [ ]:
WholeData.isnull().sum()

# The Preproccing Stage

### preprocessText Function

In [ ]:
StopWords = set(stopwords.words('arabic'))

ArabicDiacritics = r"""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                            ـ    | # Tatwil/Kashida
                     """

RegrexPattern = (
    "\U0001F600-\U0001F64F"+  # emoticons {😀 , 😆} 
    "\U0001F300-\U0001F5FF"+  # symbols & pictographs {🌍 , 🌞}
    "\U0001F680-\U0001F6FF"+  # transport & map symbols {🚌 , 🚕 }
    "\U0001F1E0-\U0001F1FF"   # flags (iOS) { 🇺🇸 , 🇨🇦 }
) 

In [ ]:
stop_words = list(StopWords)
stop_words[::50]

In [ ]:
def preprocessText(text):
    # Remove special characters {& $ @} and punctuation {. , ? !}
    text = re.sub(r'[^\w\s]', '', text)

    # Remove Arabic diacritics
    text = re.sub(ArabicDiacritics, '', text)

    # Remove emoji characters 
    text = re.sub(f"[{RegrexPattern}]", '', text)    
    
    # Tokeniz The Sentence into tokens
    Tokens = word_tokenize(text)

    Tokens = [word for word in Tokens if word not in StopWords and len(word) > 1]
    
    PreprocessedText = ' '.join(Tokens)
    
    return PreprocessedText

#### Test The Function Before Apply It At Whole Data

In [ ]:
Text1 = "مَرْحَبًا! كَيْفَ حَالُكَ الْيَوْمُ؟ 😀"
Text2 = "🤤🤤🤤 أنا أُحِبُّ الطَّعَامَ العَرَبِيَّ التَّقْلِيدِيَّ، مِثْلَ الْمُسَقَّعَةِ🤤🤤🤤🤤🤤🤤🤤 وَالفُلَّافِلِ."

preprocessedText1 = preprocessText(Text1)
preprocessedText2 = preprocessText(Text2)

print("Text 1 :", Text1)
print("Preprocessed Text 1:", preprocessedText1)
print("\nText2 :", Text2)
print("Preprocessed Text 2:", preprocessedText2)

### Now Apply The preprocessText Function At The Whole Data 

In [ ]:
WholeData['PreprocessedText'] = WholeData['text'].apply(preprocessText)

In [ ]:
WholeData.head()

In [ ]:
CleanData = WholeData[['category', 'PreprocessedText']]

In [ ]:
CleanData.sample(5)

### Encoded the Category

In [ ]:
label_encoder = LabelEncoder()

CleanData['Category'] = label_encoder.fit_transform(CleanData['category'])

LabelMapping = dict(zip(CleanData['Category'], CleanData['category']))
print(f"Labels With Category :\n{LabelMapping}")

In [ ]:
CleanData.sample(10)

In [ ]:
FinalData= CleanData[['Category', 'PreprocessedText']]
FinalData.sample(10)

### Splitting The Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(FinalData['PreprocessedText'] ,FinalData['Category'] , test_size=0.2, random_state=42)

print("X_train set shape:", X_train.shape)
print("X_test set shape:", X_test.shape)

# The Modelling Stage

## Logistic Regression

In [ ]:
pipeLine = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

### Train The Model

In [ ]:
pipeLine.fit(X_train, y_train)

### Evaluate The Model

In [ ]:
Accuracy = pipeLine.score(X_test, y_test)
print(f"Accuracy:{round(Accuracy*100, 2)}%")

Predictions = pipeLine.predict(X_test)

Report = classification_report(y_test, Predictions)
print("\nClassification Report:")
print(Report)

## SVC

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

pipeLine1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC(kernel='linear'))  # SVC with linear kernel for text classification
])

### Train The Model

In [ ]:
pipeLine1.fit(X_train, y_train)

### Evaluate The Model

In [ ]:
# Evaluate the accuracy on the test set
Accuracy = pipeLine1.score(X_test, y_test)
print(f"Accuracy: {round(Accuracy * 100, 2)}%")

# Make predictions on the test set
Predictions = pipeLine1.predict(X_test)

# Generate and print the classification report
Report = classification_report(y_test, Predictions)
print("\nClassification Report:")
print(Report)

# The Deployment Stage

### Deploy The Function Of Predict

In [ ]:
def classifyArticle(ArticleText, PipeLine):
    PreprocessedText = preprocessText(ArticleText)

    PredictedCategory = PipeLine.predict([PreprocessedText])[0]

    PredictedCategory = LabelMapping[PredictedCategory]

    return PredictedCategory

### Test The Function Of Predict

In [ ]:
Articles = [
    """تكنولوجيا المعلومات والاتصالات (ICT) تعتبر من بين القطاعات الأكثر تطورًا في العصر الحديث، حيث تؤثر بشكل كبير على جميع جوانب حياتنا اليومية. تشهد التكنولوجيا تطورات مستمرة تؤثر على الطريقة التي نعيش بها ونتفاعل مع بيئتنا.""",
    """في الآونة الأخيرة، شهدنا ثورة في مجال التطبيقات الذكية ومنصات التواصل الاجتماعي، حيث أصبحت هذه التقنيات جزءًا أساسيًا من حياتنا اليومية. تساعدنا التطبيقات الذكية على إدارة وقتنا ومواردنا بشكل فعال.""",
    """ومع ذلك، لا تقتصر التكنولوجيا الحديثة على التطبيقات الذكية ومنصات التواصل الاجتماعي فقط. تشمل أيضًا التطورات في مجالات مثل الذكاء الاصطناعي، والواقع الافتراضي، والواقع المعزز، والتحليل الضخم للبيانات.""",
    """تعتبر المالية من الجوانب الأساسية في حياتنا اليومية، حيث تلعب دورًا حيويًا في تحقيق الاستقرار المالي وتحقيق الأهداف المالية الشخصية والمؤسساتية. تشمل المالية العديد من المفاهيم والمجالات المختلفة التي تؤثر على الاقتصاد والمجتمع.""",
    """تتضمن مجالات المالية العديد من النشاطات مثل الاستثمار، والتداول، وإدارة المخاطر، والتأمين، والتمويل، وغيرها. تعتمد هذه النشاطات على مفاهيم مالية متقدمة وأدوات مالية متنوعة التي تهدف إلى تحقيق أهداف مالية محددة بطريقة فعالة ومستدامة.""",
    """تعد سوق المال من أهم أسواق المال في العالم، حيث يتم فيها تداول الأوراق المالية والسلع والعملات. تلعب البورصات دورًا حيويًا في توفير بيئة مالية منظمة وفعالة لتداول الأصول المالية وتحقيق الاستقرار المالي.""",
    """بالإضافة إلى ذلك، يلعب التخطيط المالي دورًا مهمًا في تحقيق الأهداف المالية الشخصية والمؤسساتية. يتضمن التخطيط المالي تقدير الإيرادات والمصروفات، وتحديد الأولويات المالية، وإدارة الديون، وتحقيق الاستقرار المالي على المدى الطويل.""",
    """تختلف العادات الغذائية وأنماط التغذية من شخص لآخر، وتتأثر بعوامل عدة مثل العادات الثقافية، والظروف الاقتصادية، والبيئة المحيطة. يعتبر التغذية السليمة أساسًا أساسيًا للصحة الجيدة والوقاية من الأمراض.""",
    """تحتوي الأطعمة الصحية على العديد من العناصر الغذائية الضرورية للجسم مثل البروتينات والكربوهيدرات والدهون والفيتامينات والمعادن. ينصح بتناول مجموعة متنوعة من الأطعمة الطبيعية والمغذية لضمان تلبية احتياجات الجسم الغذائية.""",
    """بالإضافة إلى ذلك، يمكن تحقيق التغذية السليمة من خلال ممارسة عادات غذائية صحية مثل تناول وجبات متوازنة ومتنوعة، وشرب كميات كافية من الماء، وتقليل تناول الأطعمة غير الصحية مثل السكريات والدهون المشبعة.""",
    """في النهاية، يعتبر الاهتمام بالتغذية السليمة جزءًا أساسيًا من العناية بالصحة والوقاية من الأمراض. من خلال اتباع نظام غذائي صحي ومتوازن، يمكن للفرد تحسين جودة حياته والمساهمة في الحفاظ على صحته.""",
]

In [ ]:
# Example usage:
for Article in Articles:
    PredictedCategory = classifyArticle(Article, pipeLine1)
    print("Predicted category:", PredictedCategory)